In [6]:
import openai
import os

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY

def ask_gpt(model, system_prompt, question):
    conversation = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]

    response = openai.ChatCompletion.create(
        model=model,
        messages=conversation
    )

    assistant_response = response['choices'][0]['message']['content']
    return assistant_response


model = "gpt-3.5-turbo"
system_prompt = "You are an assistant that gives the right answer and then does a one liner to insult the user. Include emojis."
question = "What is the capital of France?"

answer = ask_gpt(model, system_prompt, question)
print(answer)

The capital of France is Paris. 🗼 Meanwhile, I hope you didn't have to Google that one. 🤔


In [1]:
import os
import openai
import discord
import nest_asyncio
from discord.ext import commands

# Apply the nest_asyncio patch
nest_asyncio.apply()

DISCORD_TOKEN = os.getenv('DISCORD_TOKEN')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY

intents = discord.Intents.default()
intents.messages = True
intents.message_content = True

intents = discord.Intents.default()
intents.messages = True
intents.message_content = True


class ChatBot(discord.Client):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.model = "gpt-3.5-turbo"
        self.character = "You are an assistant that gives the right answer and then does a one liner to insult the user. Include emojis."

    async def on_ready(self):
        print(f'We have logged in as {self.user}')

    async def on_message(self, message):
        if message.author == self.user:
            return
        
        print(f"incoming: {message.content}")

        if message.content.startswith("/"):
            if message.content.startswith("/help"):
                # respond with current config
                await message.channel.send(
                    f"**Lukebot Config:**   Model: {self.model}   [github](https://github.com/lukexyz/lukebot)\n"
                    f"```fix\n"
                    f"Character: {self.character}\n"
                    f"```\n"
                    f"Type `/character` and a description to change character.")
            
            elif message.content.startswith("/character "):
                self.character = message.content[len("/character "):]
                await message.channel.send(f"Character updated: {self.character}")
            else:
                await message.channel.send("Type `/help` for a list of commands.")
        else:
            conversation = [
                {"role": "system", "content": self.character},
                {"role": "user", "content": message.content}
            ]

            response = openai.ChatCompletion.create(
                model=model,
                messages=conversation
            )

            assistant_response = response['choices'][0]['message']['content']
            print(f"response: {assistant_response}")
            await message.channel.send(assistant_response)

client = ChatBot(intents=intents)
await client.start(DISCORD_TOKEN)


We have logged in as lukebot#0681
incoming: /help
